In [1]:
from dataloader.BSD500 import BSD500
import torch
from R_network_relax import RNet
import json
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

In [2]:
from skimage.metrics import peak_signal_noise_ratio as compare_psnr

def compute_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = compare_psnr(Iclean[0,:,:,:], Img[0,:,:,:], data_range=data_range)
    return PSNR
    

In [3]:
test_dataset = BSD500('data/test.h5')

In [4]:
sigma = 5  


val_data = pd.read_csv(f"{'denoise_val_results'}/validation_scores_{'sigma_5_relaxed_valid'}.csv").reset_index(drop=True)
p1 = val_data.loc[val_data["psnr"].idxmax()]["p1"]


exp = "/home/pourya/mm_final/exps/sigma5/64_2_7_1e-3_relaxed"
path_ckp = exp + "/checkpoints/checkpoint_best_epoch.pth"
path_config = exp + "/config.json"
device = 'cuda:0'
config = json.load(open(path_config))
ckp = torch.load(path_ckp, map_location={'cuda:0':device,'cuda:1':device,'cuda:2':device,'cuda:3':device})

model = RNet(config['model_params'])
model.to(device)
model.load_state_dict(ckp['state_dict'])
model.eval()


model.eigenimage = model.eigenimage.to(device)

model.W1.W1.weight.data = model.zeromean(model.W1.W1.weight)
for i, _ in enumerate(model.W1.W1s): 
    model.W1.W1s[i].weight.data = model.zeromean(model.W1.W1s[i].weight)

In [5]:
i = 0
psnrs = list()

with torch.no_grad():
    for img in test_dataset:
        i = i + 1
        if True:
            gt = img.to(device)[None, :, :, :]
            noisy_image = (img.to(device) + (5.0/255.0)*torch.randn(img.shape, device=device))[None, :, :, :]
            denoised = model.solve_majorize_minimize(noisy_image)
            psnr = np.round(compute_PSNR(denoised, gt, 1), 2)
            psnrs.append(psnr)
            print(i, psnr)

1 38.42
2 37.13
3 39.57
4 38.29
5 38.55
6 36.45
7 38.21
8 37.85
9 39.23
10 37.97
11 37.31
12 36.06
13 38.21
14 36.29
15 37.5
16 39.09
17 37.03
18 37.25
19 38.65
20 38.15
21 41.05
22 35.44
23 35.56
24 35.53
25 37.76
26 38.07
27 37.33
28 40.72
29 36.73
30 37.94
31 37.13
32 39.15
33 38.24
34 34.82
35 37.6
36 37.6
37 37.66
38 37.08
39 40.66
40 35.82
41 37.85
42 41.24
43 38.54
44 39.27
45 36.31
46 38.29
47 37.73
48 36.19
49 35.22
50 45.28
51 37.2
52 39.81
53 41.52
54 39.01
55 37.82
56 36.66
57 36.42
58 36.68
59 34.98
60 36.21
61 37.18
62 36.98
63 38.31
64 35.04
65 38.44
66 41.47
67 40.27
68 39.05


In [6]:
print(np.round(np.mean(np.array(psnrs)), 2), np.round(np.std(np.array(psnrs)), 2))

37.91 1.82
